# Titanic example

## Main imports

In [1]:
import pandas as pd

import recipipe as r

## Dowload data

In this example we are going to use the Titanic dataset.
Train and test CSVs should be in `data/titanic/*.csv`.

You can download it from the Kaggle web page (https://www.kaggle.com/c/titanic/data) or, if you have the Kaggle CLI installed (https://github.com/Kaggle/kaggle-api), you can execute the next cell.

In [2]:
%%capture

from shutil import which


if which("kaggle") is not None:
    !kaggle competitions download titanic -p data
    !unzip -o data/titanic.zip -d data/titanic
else:
    pass  # Kaggle CLI not installed :( Download the data manually.

In [3]:
!ls data/titanic

gender_submission.csv  test.csv  train.csv


## Load data

In [4]:
df_train = pd.read_csv("data/titanic/train.csv")
df_test = pd.read_csv("data/titanic/test.csv")

df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Apply transformations with Recipipe

In [5]:
pipe = r.recipipe([
    # Drop unnecessary columns.
    r.drop("PassengerId", "Ticket"),
    # Impute categorical features.
    r.impute("Embarked", strategy="most_frequent"),
    # Impute numerical features.
    r.impute("Age", strategy="mean"),
    # Categorical to numeric using one-hot.
    r.onehot("Embarked", "Pclass", "Sex"),
    # Standarize numeric features.
    r.scale("Age", "Fare"),
    # Extract deck name from the cabin.
    r.extract("Cabin", pattern=r"([A-Z])\d*", col_format="Deck"),
    #r.impute("Deck", fill_value="Unknown"),
    #r.target_encode("Deck", target="Survived"),
    # Extract the English honorific title from the name.
    r.extract("Name", pattern=["Mr\.", "Mrs\.", "Miss\."], indicator=True, col_format="Title={value}"),
])

In [6]:
pipe.fit_transform(df_train).head()

,Survived,Pclass=1,Pclass=2,Pclass=3,Title=Mr,Title=Mrs,Title=Miss,Sex=female,Sex=male,Age,SibSp,Parch,Fare,Deck,Embarked=C,Embarked=Q,Embarked=S
0,0,0.0,0.0,1.0,1,0,0,0.0,1.0,-0.592481,1,0,-0.502445,NaN,0.0,0.0,1.0
1,1,1.0,0.0,0.0,0,1,0,1.0,0.0,0.638789,1,0,0.786845,C,1.0,0.0,0.0
2,1,0.0,0.0,1.0,0,0,1,1.0,0.0,-0.284663,0,0,-0.488854,NaN,0.0,0.0,1.0
3,1,1.0,0.0,0.0,0,1,0,1.0,0.0,0.407926,1,0,0.420730,C,0.0,0.0,1.0
4,0,0.0,0.0,1.0,1,0,0,0.0,1.0,0.407926,0,0,-0.486337,NaN,0.0,0.0,1.0


In [7]:
pipe.transform(df_test).head()

,Pclass=1,Pclass=2,Pclass=3,Title=Mr,Title=Mrs,Title=Miss,Sex=female,Sex=male,Age,SibSp,Parch,Fare,Deck,Embarked=C,Embarked=Q,Embarked=S
0,0.0,0.0,1.0,1,0,0,0.0,1.0,0.369449,0,0,-0.490783,NaN,0.0,1.0,0.0
1,0.0,0.0,1.0,0,1,0,1.0,0.0,1.331378,1,0,-0.507479,NaN,0.0,0.0,1.0
2,0.0,1.0,0.0,1,0,0,0.0,1.0,2.485693,0,0,-0.453367,NaN,0.0,1.0,0.0
3,0.0,0.0,1.0,1,0,0,0.0,1.0,-0.207709,0,0,-0.474005,NaN,0.0,0.0,1.0
4,0.0,0.0,1.0,0,1,0,1.0,0.0,-0.592481,1,1,-0.401017,NaN,0.0,0.0,1.0


## Custom Recipipe transformers

In [8]:
class AddColumnGroup(r.ColumnGroupsTransformer):
    def _transform_group(self, df, group_cols):
        return df[group_cols].sum(axis=1)


class ConcatColumnGroup(r.ColumnGroupsTransformer):

    def __init__(self, *args, separator="", cols_init=None, **kwargs):
        print(args)
        super().__init__(*args, **kwargs)
        self.separator = separator
        print(self.col_groups_init)
        print(self.cols_init)

    def _transform_group(self, df, group_cols):
        print("Cols", self.cols)
        return df[group_cols].astype(str).agg(self.separator.join, axis=1)

# Class aliases.
add = AddColumnGroup
concat = ConcatColumnGroup

You can add transformers to you pipeline at anytime.

In [9]:
pipe += add("SibSp", "Parch", col_format="FamilyOnboard")

Fit and transform again the same pipe with the extra transformers.

In [10]:
pipe.fit_transform(df_train)

,Survived,Pclass=1,Pclass=2,Pclass=3,Title=Mr,Title=Mrs,Title=Miss,Sex=female,Sex=male,Age,FamilyOnboard,Fare,Deck,Embarked=C,Embarked=Q,Embarked=S
0,0,0.0,0.0,1.0,1,0,0,0.0,1.0,-0.592481,1,-0.502445,NaN,0.0,0.0,1.0
1,1,1.0,0.0,0.0,0,1,0,1.0,0.0,0.638789,1,0.786845,C,1.0,0.0,0.0
2,1,0.0,0.0,1.0,0,0,1,1.0,0.0,-0.284663,0,-0.488854,NaN,0.0,0.0,1.0
3,1,1.0,0.0,0.0,0,1,0,1.0,0.0,0.407926,1,0.420730,C,0.0,0.0,1.0
4,0,0.0,0.0,1.0,1,0,0,0.0,1.0,0.407926,0,-0.486337,NaN,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,0.0,1.0,0.0,0,0,0,0.0,1.0,-0.207709,0,-0.386671,NaN,0.0,0.0,1.0
887,1,1.0,0.0,0.0,0,0,1,1.0,0.0,-0.823344,0,-0.044381,B,0.0,0.0,1.0
888,0,0.0,0.0,1.0,0,0,1,1.0,0.0,0.000000,3,-0.176263,NaN,0.0,0.0,1.0
889,1,1.0,0.0,0.0,1,0,0,0.0,1.0,-0.284663,0,-0.044381,C,1.0,0.0,0.0


In [11]:
pipe.transform(df_test)

,Pclass=1,Pclass=2,Pclass=3,Title=Mr,Title=Mrs,Title=Miss,Sex=female,Sex=male,Age,FamilyOnboard,Fare,Deck,Embarked=C,Embarked=Q,Embarked=S
0,0.0,0.0,1.0,1,0,0,0.0,1.0,0.369449,0,-0.490783,NaN,0.0,1.0,0.0
1,0.0,0.0,1.0,0,1,0,1.0,0.0,1.331378,1,-0.507479,NaN,0.0,0.0,1.0
2,0.0,1.0,0.0,1,0,0,0.0,1.0,2.485693,0,-0.453367,NaN,0.0,1.0,0.0
3,0.0,0.0,1.0,1,0,0,0.0,1.0,-0.207709,0,-0.474005,NaN,0.0,0.0,1.0
4,0.0,0.0,1.0,0,1,0,1.0,0.0,-0.592481,2,-0.401017,NaN,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.0,0.0,1.0,1,0,0,0.0,1.0,0.000000,0,-0.486337,NaN,0.0,0.0,1.0
414,1.0,0.0,0.0,0,0,0,1.0,0.0,0.715743,0,1.544246,C,1.0,0.0,0.0
415,0.0,0.0,1.0,1,0,0,0.0,1.0,0.677266,0,-0.502445,NaN,0.0,0.0,1.0
416,0.0,0.0,1.0,1,0,0,0.0,1.0,0.000000,0,-0.486337,NaN,0.0,0.0,1.0


## What's next?

You can continue learning about Recipipe by:

* Following this [fun tutorial about Paranoids](https://github.com/guiferviz/recipipe/blob/master/examples/paranoids.ipynb).
* Reading the [official documentation](https://guiferviz.com/recipipe/).
* Reading the source code from the [GitHub repository](https://github.com/guiferviz/recipipe) :).